In [1]:
import os
import sys
sys.path.append("..")
import phage_init

In [2]:
import numpy
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy import stats
from Bio.Alphabet import IUPAC
#from itertools import permutations


In [ ]:
#from numba import jit,vectorize

In [3]:
all_fasta=(os.path.join(phage_init.fasta_dir_2,"major_capsid_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"minor_capsid_all_clustered.fasta"),
           os.path.join(phage_init.fasta_dir_2,"baseplate_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"major_tail_all_clustered.fasta"),
           os.path.join(phage_init.fasta_dir_2,"minor_tail_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"portal_all_clustered.fasta"),
           os.path.join(phage_init.fasta_dir_2,"tail_fiber_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"shaft_all_clustered.fasta"),
           os.path.join(phage_init.fasta_dir_2,"collar_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"HTJ_all_clustered.fasta"),
           os.path.join(phage_init.fasta_dir_2,"others_all_clustered.fasta"))

In [4]:

def extract_all2(fasta_list):
    AA=["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]
    di_pep = [''.join(i) for i in itertools.product(AA, repeat = 2)]
    AAs=set(AA)
    prot_class=0;
    arr = numpy.empty((0,408), dtype=numpy.float)
    class_arr = numpy.empty((0), dtype=numpy.int)
    for file in fasta_list:
        print('####################' + file)
        for record in SeqIO.parse(file, "fasta"):
            ll=len(record.seq)
            seqq=record.seq.__str__().upper()
            seqqq=seqq.replace('X','A').replace('J','L').replace('*','A').replace('Z','E').replace('B','D')
            X = ProteinAnalysis(seqqq)
#            if (bool(set(seqqq)-AAs)):
#                print(record.description)
#                print(seqqq)
            tt= [X.isoelectric_point(), X.instability_index(),ll,X.aromaticity(),
                X.molar_extinction_coefficient()[0],X.molar_extinction_coefficient()[1],
                X.gravy(),X.molecular_weight()]
            tt_n = numpy.asarray(tt,dtype=numpy.float)
            
            di_pep_count=[seqq.count(i)/(ll-1) for i in di_pep]
            di_pep_count_n = numpy.asarray(di_pep_count,dtype=numpy.float)
    
            cat_n= numpy.concatenate((di_pep_count_n,tt_n))
            #print(cat_n.shape)
            cat_n = cat_n.reshape((1,cat_n.shape[0]))

            arr = numpy.append(arr,cat_n , axis=0)
            class_arr = numpy.append(class_arr,prot_class)
        prot_class+=1
    return (arr,class_arr)
    

In [ ]:
(arr,class_arr)=extract_all2(all_fasta)
print(arr.shape)

####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/major_capsid_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/minor_capsid_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/baseplate_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/major_tail_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/minor_tail_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/portal_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/tail_fiber_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/

In [ ]:
print(arr.shape)

In [ ]:

arr_z=numpy.apply_along_axis(stats.zscore,0,arr)



In [ ]:
nb_classes = 11
one_hot_targets = numpy.eye(nb_classes)[class_arr]

In [ ]:
final = numpy.concatenate((arr_z, one_hot_targets), axis=1)

In [ ]:
mean_arr=numpy.apply_along_axis(numpy.mean,0,arr)
#print(mean_arr)
print(mean_arr.shape)
std_arr=numpy.apply_along_axis(numpy.std,0,arr)
print(std_arr.shape)

In [ ]:
numpy.random.shuffle(final)

In [ ]:
#import pickle
##number of proteins for training
tt=50000
##number of features
f_num=arr.shape[1]
#pickle.dump(final[0:tt,0:f_num], open( os.path.join(phage_init.data_dir,"train_x_data2.p"), "wb" ) )
#pickle.dump(final[tt:,0:f_num],  open( os.path.join(phage_init.data_dir,"test_x_data2.p") , "wb" ) )
#pickle.dump(final[0:tt,f_num:], open( os.path.join(phage_init.data_dir,"train_y_data2.p"), "wb" ) )
#pickle.dump(final[tt:,f_num:],  open( os.path.join(phage_init.data_dir,"test_y_data2.p") , "wb" ) )
#pickle.dump(mean_arr,  open( os.path.join(phage_init.data_dir,"mean_data2.p") , "wb" ) )
#pickle.dump(std_arr,  open( os.path.join(phage_init.data_dir,"std_data2.p") , "wb" ) )

In [ ]:
train_x=final[0:tt,0:f_num]
train_y=final[0:tt,f_num:]
test_x=final[tt:,0:f_num]
test_y=final[tt:,f_num:]
              

In [ ]:
#import keras and numpy
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import Adam

In [ ]:
#this list the devices, just making sure there is a GPU present, you might be fine with no GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
model = Sequential()
opt=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.add(Dense(f_num, input_dim=f_num, kernel_initializer='random_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(11,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())
